# Step 2: Create Incident with Alert - **Validated Implementation**

This notebook creates a comprehensive **phishing incident** using validated implementation patterns. It demonstrates:

- **Incident Context Creation**: Establish incident-specific context storage using confirmed patterns
- **Phishing Alert Objects**: Create STIX objects for phishing email evidence using dual-layer format
- **Observable Evidence**: Document email addresses, URLs, and file attachments as cyber observables
- **Incident Relationships**: Link evidence objects to the incident investigation
- **Context Memory Integration**: Store all objects in incident-specific context using validated storage patterns

## 🎯 Phishing Incident Scenario

**Incident Type**: Phishing email with malicious URL and attachment
**Reporter**: Company employee who received suspicious email
**Evidence**: Sender email, malicious URL, suspicious attachment
**Investigation Status**: New incident requiring analysis

## 📁 Context Memory Structure Created

```
context_mem/
├── context_map.json                    # Global context routing
└── incident--{incident-uuid}/          # Incident-specific context
    ├── incident.json                   # Primary incident object
    ├── observables.json                # Email addresses, URLs, files
    ├── indicators.json                 # Threat indicators
    ├── relationships.json              # Object relationships
    └── evidence.json                   # Investigation evidence
```

## ✅ Prerequisites

- **User Context**: Must run `Step_0_User_Setup.ipynb` first
- **Company Context**: Must run `Step_1_Company_Setup.ipynb` second
- **Template Files**: Incident and observable templates must exist

## A.1 Environment Setup

Initialize STIX libraries and incident-specific extensions for comprehensive phishing incident modeling.

In [62]:
# Install required STIX libraries for incident modeling
import sys
!{sys.executable} -m pip install stixorm

# Import core STIX 2.1 objects for incident management
from stixorm.module.authorise import import_type_factory
from stixorm.module.definitions.stix21 import (
    Identity, EmailAddress, UserAccount, Relationship, Bundle, 
    Incident, Indicator, ObservedData, URL, File
)

# Import processing utilities and result handling
from stixorm.module.typedb_lib.instructions import ResultStatus, Result
from stixorm.module.parsing import parse_objects

# Initialize import factory for STIX object creation
import_type = import_type_factory.get_all_imports()

# Setup logging for incident operations
import logging
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

print("✅ STIX libraries initialized for incident modeling")
print("✅ Core STIX 2.1 objects loaded for phishing scenarios")
print("✅ Logging configured for incident operations")

✅ STIX libraries initialized for incident modeling
✅ Core STIX 2.1 objects loaded for phishing scenarios
✅ Logging configured for incident operations



[notice] A new release of pip is available: 24.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


## A.2 Path Configuration

Configure Python paths and working directory for incident operations using validated path resolution patterns.

In [63]:
# Configure Python path for relative imports
import sys
sys.path.append('../')
import os

# Get current working directory for incident operations
cwd = os.getcwd()

print(f"✅ Working directory: {cwd}")
print("✅ Python path configured for incident context operations")

✅ Working directory: c:\projects\brett_blocks\Orchestration
✅ Python path configured for incident context operations


## A.3 Import Incident Utilities and Configure Phishing Scenario

Import validated utility functions for incident context management and configure the phishing incident scenario with realistic attack data.

In [64]:
import json

# Import validated utility functions for incident management
from Utilities.local_make_general import (
    invoke_create_incident_context, 
    invoke_save_incident_context_block,
    invoke_get_from_company_block,
    invoke_get_from_user_block
)

# Import STIX object creation utilities  
from Utilities.local_make_sdo import (
    invoke_make_incident_block,
    invoke_make_indicator_block,
    invoke_make_observed_data_block,  # ✅ Added: Missing ObservedData creation
    invoke_make_event_block,          # ✅ Added: Missing Event creation
    invoke_make_sequence_block,       # ✅ Added: Missing Sequence creation
    invoke_make_task_block            # ✅ Added: Missing Task creation
)

from Utilities.local_make_sco import (
    invoke_make_email_addr_block,
    invoke_make_url_block,
    invoke_make_e_msg_block          # ✅ Added: Missing Email Message creation
)

from Utilities.local_make_sro import invoke_sro_block, invoke_sighting_block  # ✅ Added: Missing Sighting creation

# Import utilities for data processing
from Utilities.util import emulate_ports, unwind_ports, conv

# Configure base paths using validated patterns
path_base = "../Block_Families/StixORM/"  # ✅ Validated structure
results_base = "../Orchestration/Results/"
context_base = "../Orchestration/generated/os-triage/context_mem/"

# Define phishing incident scenario data
phishing_scenario = {
    "incident_name": "Phishing Email Investigation",
    "incident_type": "phishing-email",
    "investigation_status": "new",
    "reporter_email": "alice.jones@company.com",  # Employee who reported
    "attacker_email": "security-update@bankofamerica-verify.com",  # Spoofed sender
    "malicious_url": "https://bankofamerica-verify.com/login-verify.php",
    "attachment_name": "account_verification.pdf",
    "attachment_hash": "d41d8cd98f00b204e9800998ecf8427e",
    "subject": "Urgent: Verify Your Account - Security Alert",
    "description": "Sophisticated phishing attempt targeting employee with fake bank security alert"
}

print("✅ Incident utilities imported successfully")
print(f"✅ Base paths configured:")
print(f"   - Objects: {path_base}")
print(f"   - Results: {results_base}")
print(f"   - Context: {context_base}")
print(f"✅ Phishing incident scenario configured:")
print(f"   - Type: {phishing_scenario['incident_type']}")
print(f"   - Reporter: {phishing_scenario['reporter_email']}")
print(f"   - Attacker: {phishing_scenario['attacker_email']}")
print(f"   - Malicious URL: {phishing_scenario['malicious_url']}")

✅ Incident utilities imported successfully
✅ Base paths configured:
   - Objects: ../Block_Families/StixORM/
   - Results: ../Orchestration/Results/
   - Context: ../Orchestration/generated/os-triage/context_mem/
✅ Phishing incident scenario configured:
   - Type: phishing-email
   - Reporter: alice.jones@company.com
   - Attacker: security-update@bankofamerica-verify.com
   - Malicious URL: https://bankofamerica-verify.com/login-verify.php


## B. Create the Phishing Incident

Initialize the primary incident object for the phishing investigation. This creates:

1. **Incident Object**: Primary STIX incident with phishing classification
2. **Incident Context**: Dedicated context directory for investigation data
3. **Context Routing**: Updates global context map for incident tracking
4. **Investigation Framework**: Foundation for evidence collection and analysis

In [65]:
# Configure incident creation paths - use working template from original
incident_template_path = "SDO/Incident/phishing_incident.json"  # ✅ Use working template
incident_results_path = "step1/phishing_incident"  # ✅ Fixed: Use step1 like OLD notebook

print(f"🔍 Creating phishing incident: {phishing_scenario['incident_name']}")

# Initialize empty object reference lists for the incident
sequence_start_refs = []  # Investigation sequence starting points
sequence_refs = []        # Investigation sequence steps
task_refs = []           # Investigation tasks
event_refs = []          # Security events
impact_refs = []         # Business impact objects
other_object_refs = []   # Evidence and observable objects

# Create the primary incident object using validated utility pattern
incident_obj = invoke_make_incident_block(
    incident_template_path,
    incident_results_path,
    sequence_start_refs,
    sequence_refs, 
    task_refs,
    event_refs,
    impact_refs,
    other_object_refs
)

# Configure context type for incident storage
context_type = {
    "context_type": "incident"  # ✅ Required for incident context
}

# Define context storage paths - match working pattern from Step 1
incident_results_obj_path = results_base + incident_results_path  # The incident JSON file
incident_results_context_path = results_base + "step1/incident_context.json"  # ✅ Fixed: Use step1

# Create incident context directory and save incident object
result = invoke_create_incident_context(incident_results_obj_path, incident_results_context_path)

print(f"✅ Phishing incident created successfully")
print(f"   - Incident object: {incident_results_obj_path}")
print(f"   - Context storage: {incident_results_context_path}")
print(f"   - Context creation result: {result}")
print(f"🔍 Incident context is now active and ready for evidence collection")

🔍 Creating phishing incident: Phishing Email Investigation
{
    "type": "incident",
    "spec_version": "2.1",
    "id": "incident--d7aaa480-7a3e-4141-8fe0-baaeb6e93f24",
    "created": "2025-10-29T14:55:21.889Z",
    "modified": "2025-10-29T14:55:21.889Z",
    "name": "potential phishing",
    "extensions": {
        "extension-definition--ef765651-680c-498d-9894-99799f2fa126": {
            "extension_type": "property-extension",
            "investigation_status": "new",
            "incident_types": [
                "dissemination-phishing-emails"
            ]
        }
    }
}
✅ Phishing incident created successfully
   - Incident object: ../Orchestration/Results/step1/phishing_incident
   - Context storage: ../Orchestration/Results/step1/incident_context.json
   - Context creation result: {'context_result': ' incident context created -> incident--d7aaa480-7a3e-4141-8fe0-baaeb6e93f24\nstix_id -> incident--d7aaa480-7a3e-4141-8fe0-baaeb6e93f24'}
🔍 Incident context is now active a

## C. Create Phishing Email Evidence Objects

Document the phishing email evidence using STIX cyber observable objects. This creates comprehensive evidence documentation for the investigation.

### C.1 Create Attacker Email Address

Document the spoofed email address used by the attacker in the phishing attempt.

In [66]:
# Configure attacker email address creation - use working template
attacker_email_template = "SCO/Email_Addr/email_addr_THREAT.json"  # ✅ Use working template
attacker_email_results = "step2/attacker_email"

print(f"📧 Creating attacker email address evidence: {phishing_scenario['attacker_email']}")

# Create the attacker email address object
attacker_email_obj = invoke_make_email_addr_block(
    attacker_email_template,
    attacker_email_results
)

# Configure context type for evidence storage
context_type = {
    "context_type": "observables"  # ✅ Store in observables.json
}

# Define storage paths for attacker email evidence
attacker_email_obj_path = results_base + attacker_email_results + "__email.json"
attacker_email_context_path = results_base + "step2/context/attacker_email_context.json"

# Save attacker email to incident context
result = invoke_save_incident_context_block(
    attacker_email_obj_path,
    attacker_email_context_path,
    context_type
)

print(f"✅ Attacker email address documented successfully")
print(f"   - Evidence object: {attacker_email_obj_path}")
print(f"   - Context storage result: {result}")

📧 Creating attacker email address evidence: security-update@bankofamerica-verify.com
{
    "type": "email-addr",
    "spec_version": "2.1",
    "id": "email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf212",
    "value": "evil@northkorea.nk",
    "display_name": "Bad Man"
}
input data->{'stix_object': {'type': 'email-addr', 'spec_version': '2.1', 'id': 'email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf212', 'value': 'evil@northkorea.nk', 'display_name': 'Bad Man'}, 'context_type': {'context_type': 'observables'}}
from ports 
stix_object->{'type': 'email-addr', 'spec_version': '2.1', 'id': 'email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf212', 'value': 'evil@northkorea.nk', 'display_name': 'Bad Man'}
context type->observables
✅ Attacker email address documented successfully
   - Evidence object: ../Orchestration/Results/step2/attacker_email__email.json
   - Context storage result: {'context_result': ' incident context saved - \nstix_id -> email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf212'}
✅ Attacker

### C.2 Create Malicious URL Evidence

Document the malicious URL contained in the phishing email that attempts to steal credentials.

In [67]:
# Configure malicious URL creation - use working template
malicious_url_template = "SCO/URL/suspicious_url.json"  # ✅ Use working template
malicious_url_results = "step2/malicious_url.json"  # ✅ Fixed: Add .json extension like OLD notebook

print(f"🌐 Creating malicious URL evidence: {phishing_scenario['malicious_url']}")

# Create the malicious URL object with required URL value (follow OLD notebook pattern)
malicious_url_obj = invoke_make_url_block(
    malicious_url_template,
    malicious_url_results,
    phishing_scenario['malicious_url']  # ✅ Fixed: Provide the URL value
)

# Configure context type for evidence storage
context_type = {
    "context_type": "observables"  # ✅ Store in observables.json
}

# Define storage paths for malicious URL evidence (match OLD notebook pattern)
malicious_url_obj_path = results_base + malicious_url_results  # ✅ Fixed: Use exact file path
malicious_url_context_path = results_base + "step2/context/malicious_url_context.json"

# Save malicious URL to incident context
result = invoke_save_incident_context_block(
    malicious_url_obj_path,
    malicious_url_context_path,
    context_type
)

print(f"✅ Malicious URL documented successfully")
print(f"   - Evidence object: {malicious_url_obj_path}")
print(f"   - Context storage result: {result}")

🌐 Creating malicious URL evidence: https://bankofamerica-verify.com/login-verify.php
{
    "type": "url",
    "spec_version": "2.1",
    "id": "url--a824ef3f-83a0-51ae-9b5b-817580ec74ee",
    "value": "https://bankofamerica-verify.com/login-verify.php"
}
input data->{'stix_object': {'type': 'url', 'spec_version': '2.1', 'id': 'url--a824ef3f-83a0-51ae-9b5b-817580ec74ee', 'value': 'https://bankofamerica-verify.com/login-verify.php'}, 'context_type': {'context_type': 'observables'}}
from ports 
stix_object->{'type': 'url', 'spec_version': '2.1', 'id': 'url--a824ef3f-83a0-51ae-9b5b-817580ec74ee', 'value': 'https://bankofamerica-verify.com/login-verify.php'}
context type->observables
✅ Malicious URL documented successfully
   - Evidence object: ../Orchestration/Results/step2/malicious_url.json
   - Context storage result: {'context_result': ' incident context saved - \nstix_id -> url--a824ef3f-83a0-51ae-9b5b-817580ec74ee'}


### C.3 Create Malicious Attachment Evidence (Future Enhancement)

Due to template limitations, file attachment evidence creation is deferred to future enhancement. The investigation continues with email and URL evidence which provides sufficient basis for phishing incident analysis.

```python
# TODO: Implement file attachment evidence when File block templates are available
# malicious_file_template = "SCO/File/File_template.json"
# malicious_file_results = "step2/malicious_attachment"
print(f"📎 File attachment evidence planned: {phishing_scenario['attachment_name']}")
print(f"   - File hash: {phishing_scenario['attachment_hash']}")
print(f"⚠️ File evidence creation deferred - email and URL evidence sufficient for investigation")
```

In [68]:
# TODO: Implement file attachment evidence when File block templates are available
# malicious_file_template = "SCO/File/File_template.json"
# malicious_file_results = "step2/malicious_attachment"
print(f"📎 File attachment evidence planned: {phishing_scenario['attachment_name']}")
print(f"   - File hash: {phishing_scenario['attachment_hash']}")
print(f"⚠️ File evidence creation deferred - email and URL evidence sufficient for investigation")

📎 File attachment evidence planned: account_verification.pdf
   - File hash: d41d8cd98f00b204e9800998ecf8427e
⚠️ File evidence creation deferred - email and URL evidence sufficient for investigation


## D. Create Threat Indicators

Generate STIX Indicator objects that represent patterns of malicious activity for threat detection and prevention.

### D.1 Create Email Domain Indicator

Create an indicator for the malicious email domain used in the phishing attack.

In [69]:
# Configure email domain indicator creation - use working template
email_indicator_template = "SDO/Indicator/indicator_alert.json"  # ✅ Use working template
email_indicator_results = "step2/email_domain_indicator.json"  # ✅ Fixed: Add .json extension

print(f"🚨 Creating email domain threat indicator")
print(f"   - Malicious domain: bankofamerica-verify.com")

# Create the email domain indicator object (follow OLD notebook pattern)
email_indicator_obj = invoke_make_indicator_block(
    email_indicator_template,
    email_indicator_results,
    pattern="[email-addr:value = 'security-update@bankofamerica-verify.com']"  # ✅ Fixed: Use exact match like OLD notebook
)

# Configure context type for indicator storage
context_type = {
    "context_type": "indicators"  # ✅ Store in indicators.json
}

# Define storage paths for email indicator (match OLD notebook pattern)
email_indicator_obj_path = results_base + email_indicator_results  # ✅ Fixed: Use exact file path
email_indicator_context_path = results_base + "step2/context/email_indicator_context.json"

# Save email indicator to incident context
result = invoke_save_incident_context_block(
    email_indicator_obj_path,
    email_indicator_context_path,
    context_type
)

print(f"✅ Email domain indicator created successfully")
print(f"   - Indicator object: {email_indicator_obj_path}")
print(f"   - Context storage result: {result}")

🚨 Creating email domain threat indicator
   - Malicious domain: bankofamerica-verify.com
inputfile->../Block_Families/StixORM/SDO/Indicator/indicator_alert.json
path exists
input->{'indicator_form': {'base_required': {'type': 'identity', 'spec_version': '2.1', 'id': '', 'created': '', 'modified': ''}, 'base_optional': {'created_by_ref': '', 'revoked': None, 'labels': [], 'lang': '', 'external_references': [], 'object_marking_refs': [], 'granular_markings': []}, 'object': {'name': 'Potential Phishing Email', 'description': 'Suspicious email reported by user', 'pattern_type': 'stix', 'pattern': '', 'indicator_types': ['unknown']}, 'extensions': {}, 'sub': {}}, 'pattern': "[email-addr:value = 'security-update@bankofamerica-verify.com']"}
pattern [email-addr:value = 'security-update@bankofamerica-verify.com']
{
    "type": "indicator",
    "spec_version": "2.1",
    "id": "indicator--82392840-f7d6-49d3-a2f0-591459637097",
    "created": "2025-10-29T14:55:22.193Z",
    "modified": "2025-10-

### D.2 Create URL Pattern Indicator

Create an indicator for the malicious URL pattern used to steal credentials.

In [70]:
# Configure URL pattern indicator creation - use working template  
url_indicator_template = "SDO/Indicator/indicator_alert.json"  # ✅ Use working template
url_indicator_results = "step2/url_pattern_indicator.json"  # ✅ Fixed: Add .json extension

print(f"🚨 Creating URL pattern threat indicator")
print(f"   - Malicious pattern: login-verify.php")

# Create the URL pattern indicator object (follow OLD notebook pattern)
url_indicator_obj = invoke_make_indicator_block(
    url_indicator_template,
    url_indicator_results,
    pattern="[url:value = 'https://bankofamerica-verify.com/login-verify.php']"  # ✅ Fixed: Use exact match like OLD notebook
)

# Configure context type for indicator storage
context_type = {
    "context_type": "indicators"  # ✅ Store in indicators.json
}

# Define storage paths for URL indicator (match OLD notebook pattern)
url_indicator_obj_path = results_base + url_indicator_results  # ✅ Fixed: Use exact file path
url_indicator_context_path = results_base + "step2/context/url_indicator_context.json"

# Save URL indicator to incident context
result = invoke_save_incident_context_block(
    url_indicator_obj_path,
    url_indicator_context_path,
    context_type
)

print(f"✅ URL pattern indicator created successfully")
print(f"   - Indicator object: {url_indicator_obj_path}")
print(f"   - Context storage result: {result}")

🚨 Creating URL pattern threat indicator
   - Malicious pattern: login-verify.php
inputfile->../Block_Families/StixORM/SDO/Indicator/indicator_alert.json
path exists
input->{'indicator_form': {'base_required': {'type': 'identity', 'spec_version': '2.1', 'id': '', 'created': '', 'modified': ''}, 'base_optional': {'created_by_ref': '', 'revoked': None, 'labels': [], 'lang': '', 'external_references': [], 'object_marking_refs': [], 'granular_markings': []}, 'object': {'name': 'Potential Phishing Email', 'description': 'Suspicious email reported by user', 'pattern_type': 'stix', 'pattern': '', 'indicator_types': ['unknown']}, 'extensions': {}, 'sub': {}}, 'pattern': "[url:value = 'https://bankofamerica-verify.com/login-verify.php']"}
pattern [url:value = 'https://bankofamerica-verify.com/login-verify.php']
{
    "type": "indicator",
    "spec_version": "2.1",
    "id": "indicator--305e27fb-6ae6-484d-be8c-9f53fd14a352",
    "created": "2025-10-29T14:55:22.314Z",
    "modified": "2025-10-29T1

## D.3 Create Email Message Evidence

Create the email message object that links the attacker email to the malicious URL, following the OLD notebook pattern.

In [71]:
# Configure email message creation - follow OLD notebook pattern
email_msg_template = "SCO/Email_Message/suspicious_email_msg.json"
email_msg_results = "step2/phishing_email_msg.json"

print(f"📧 Creating phishing email message evidence")

# Create email message object linking attacker email and URL
from_ref = attacker_email_obj
to_ref = [reporter_email_obj] if 'reporter_email_obj' in locals() else []  # Use reporter if found
cc_ref = []
bcc_ref = []

# Create the email message object
email_msg_obj = invoke_make_e_msg_block(
    email_msg_template, 
    email_msg_results, 
    from_ref, 
    to_ref, 
    cc_ref, 
    bcc_ref
)

# Configure context type for evidence storage
context_type = {
    "context_type": "observables"  # ✅ Store in observables.json
}

# Define storage paths for email message
email_msg_obj_path = results_base + email_msg_results
email_msg_context_path = results_base + "step2/context/email_msg_context.json"

# Save email message to incident context
result = invoke_save_incident_context_block(
    email_msg_obj_path,
    email_msg_context_path,
    context_type
)

print(f"✅ Email message evidence created successfully")
print(f"   - Message object: {email_msg_obj_path}")
print(f"   - Context storage result: {result}")

📧 Creating phishing email message evidence
{
    "type": "email-message",
    "spec_version": "2.1",
    "id": "email-message--6090e3d4-1fa8-5b36-9d2d-4a66d824995d",
    "is_multipart": false,
    "date": "2020-10-19T01:01:01Z",
    "from_ref": "email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf212",
    "to_refs": [
        "email-addr--4722424c-7012-56b0-84d5-01d076fc547b"
    ],
    "subject": "we are coming for you",
    "body": "some bad stuff written here"
}
input data->{'stix_object': {'type': 'email-message', 'spec_version': '2.1', 'id': 'email-message--6090e3d4-1fa8-5b36-9d2d-4a66d824995d', 'is_multipart': False, 'date': '2020-10-19T01:01:01.000Z', 'from_ref': 'email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf212', 'to_refs': ['email-addr--4722424c-7012-56b0-84d5-01d076fc547b'], 'subject': 'we are coming for you', 'body': 'some bad stuff written here'}, 'context_type': {'context_type': 'observables'}}
from ports 
stix_object->{'type': 'email-message', 'spec_version': '2.1', 'id': 'emai

## D.4 Create ObservedData Object

Create the ObservedData object that groups all evidence objects together, following the STIX pattern from the OLD notebook.

In [72]:
# Create ObservedData object following OLD notebook pattern
observed_data_obj = invoke_make_observed_data_block(
    observed_data_template,
    observed_data_results,
    observation=observed_list
)

# Configure context type for evidence storage
context_type = {
    "context_type": "unattached"  # ✅ Store as unattached like OLD notebook
}

# Define storage paths for ObservedData
observed_data_obj_path = results_base + observed_data_results
observed_data_context_path = results_base + "step2/context/observed_data_context.json"

# Save ObservedData to incident context
result = invoke_save_incident_context_block(
    observed_data_obj_path,
    observed_data_context_path,
    context_type
)

# Create compatibility aliases for OLD notebook variable names
obs_1 = observed_data_obj  # OLD notebook compatibility

print(f"✅ ObservedData object created successfully")
print(f"   - Observed objects count: {len(observed_list)}")
print(f"   - ObservedData object: {observed_data_obj_path}")
print(f"   - Context storage result: {result}")
print(f"   - Variable alias obs_1 created for compatibility")

{
    "type": "observed-data",
    "spec_version": "2.1",
    "id": "observed-data--ee9fd827-d30f-4dde-923c-73d5e076999f",
    "created": "2025-10-29T14:55:22.525Z",
    "modified": "2025-10-29T14:55:22.525Z",
    "first_observed": "2020-10-19T01:01:01Z",
    "last_observed": "2020-10-19T01:01:01Z",
    "number_observed": 1,
    "object_refs": [
        "email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf212",
        "url--a824ef3f-83a0-51ae-9b5b-817580ec74ee",
        "email-message--6090e3d4-1fa8-5b36-9d2d-4a66d824995d"
    ]
}
input data->{'stix_object': {'type': 'observed-data', 'spec_version': '2.1', 'id': 'observed-data--ee9fd827-d30f-4dde-923c-73d5e076999f', 'created': '2025-10-29T14:55:22.525Z', 'modified': '2025-10-29T14:55:22.525Z', 'first_observed': '2020-10-19T01:01:01.000Z', 'last_observed': '2020-10-19T01:01:01.000Z', 'number_observed': 1, 'object_refs': ['email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf212', 'url--a824ef3f-83a0-51ae-9b5b-817580ec74ee', 'email-message--6090e3d4-1

## D.5 Create Sighting Object

Create the Sighting object that connects the indicators to the observed data, establishing the threat detection linkage.

In [73]:
# Configure Sighting creation - follow OLD notebook pattern
sighting_template = "SRO/Sighting/sighting_alert.json"
sighting_results = "step2/sighting_alert.json"

print(f"👁️ Creating Sighting object to link indicators to observations")

# Setup sighting parameters following OLD notebook pattern exactly
sighted = email_indicator_obj  # Primary indicator being sighted  
observation_list = [observed_data_obj]  # ObservedData containing evidence

# Try to get reporter identity from company context (identity type, not email)
try:
    reporter_identity_query = {
        "type": "identity",
        "property": {
            "path": ["name"],
            "source_value": "Naive Smith",  # Company employee from Step 1
            "comparator": "EQ"
        }
    }
    company_context_type = {"context_type": "company"}
    reporter_identity = invoke_get_from_company_block(
        reporter_identity_query, 
        company_context_type, 
        source_value=None, 
        source_id=None
    )
    where_list = [reporter_identity]  # Identity type works for where_sighted_refs
    print(f"✅ Found reporter identity: {reporter_identity.get('name', 'Unknown')}")
except Exception as e:
    where_list = []  # Empty list if no identity found
    print(f"⚠️ Reporter identity not found, proceeding without location: {str(e)}")

# Create the Sighting object
sighting_obj = invoke_sighting_block(
    sighting_template,
    sighting_results,
    observed=observation_list,
    sighted=sighted,
    where=where_list
)

👁️ Creating Sighting object to link indicators to observations
company query->{'type': 'identity', 'property': {'path': ['name'], 'source_value': 'Naive Smith', 'comparator': 'EQ'}}
✅ Found reporter identity: Example Company
<class 'dict'>
['sighting_form', 'observed_data_refs', 'where_sighted_refs', 'sighting_of_ref']
{
    "type": "sighting",
    "spec_version": "2.1",
    "id": "sighting--1685385a-7862-4141-955d-f2f990c33ede",
    "created": "2025-10-29T14:55:22.691Z",
    "modified": "2025-10-29T14:55:22.691Z",
    "count": 1,
    "sighting_of_ref": "indicator--82392840-f7d6-49d3-a2f0-591459637097",
    "observed_data_refs": [
        "observed-data--ee9fd827-d30f-4dde-923c-73d5e076999f"
    ],
    "where_sighted_refs": [
        "identity--e45d17c4-563a-4223-b75d-5b4ca180577d"
    ],
    "extensions": {
        "extension-definition--0d76d6d9-16ca-43fd-bd41-4f800ba8fc43": {
            "extension_type": "property-extension"
        },
        "sighting-alert": {
            "name"

## D.6 Create Event and Task Objects

Create Event and Task objects for the incident investigation workflow, following the OLD notebook pattern.

In [74]:
# Configure Event creation - follow OLD notebook pattern
event_template = "SDO/Event/event_alert.json"
event_results = "step2/event_alert.json"

print(f"📅 Creating Event object for incident timeline")

# Create the Event object
event_obj = invoke_make_event_block(event_template, event_results)

# Configure context type for event storage
context_type = {
    "context_type": "event"  # ✅ Store as event
}

# Define storage paths for Event
event_obj_path = results_base + event_results
event_context_path = results_base + "step2/context/event_context.json"

# Save Event to incident context
result = invoke_save_incident_context_block(
    event_obj_path,
    event_context_path,
    context_type
)

print(f"✅ Event object created successfully")
print(f"   - Event object: {event_obj_path}")
print(f"   - Context storage result: {result}")

# Configure Task creation - follow OLD notebook pattern
task_template = "SDO/Task/task_alert.json"
task_results = "step2/task_alert.json"

print(f"📋 Creating Task object for investigation workflow")

# Create the Task object
task_obj = invoke_make_task_block(task_template, task_results)

# Configure context type for task storage
context_type = {
    "context_type": "task"  # ✅ Store as task
}

# Define storage paths for Task
task_obj_path = results_base + task_results
task_context_path = results_base + "step2/context/task_context.json"

# Save Task to incident context
result = invoke_save_incident_context_block(
    task_obj_path,
    task_context_path,
    context_type
)

print(f"✅ Task object created successfully")
print(f"   - Task object: {task_obj_path}")
print(f"   - Context storage result: {result}")

📅 Creating Event object for incident timeline
{
    "type": "event",
    "spec_version": "2.1",
    "id": "event--40cb3fd1-4092-47f0-93c7-cf75d9b18f64",
    "created": "2025-10-29T14:55:22.737Z",
    "modified": "2025-10-29T14:55:22.737Z",
    "description": "Suspicious email reported by user",
    "end_time": "2025-10-29T14:55:22.737Z",
    "event_types": [
        "dissemination-phishing-emails"
    ],
    "name": "Potential Phishing Email",
    "start_time": "2025-10-29T14:55:22.737Z",
    "extensions": {
        "extension-definition--4ca6de00-5b0d-45ef-a1dc-ea7279ea910e": {
            "extension_type": "new-sdo"
        }
    }
}
input data->{'stix_object': {'type': 'event', 'spec_version': '2.1', 'id': 'event--40cb3fd1-4092-47f0-93c7-cf75d9b18f64', 'created': '2025-10-29T14:55:22.737Z', 'modified': '2025-10-29T14:55:22.737Z', 'description': 'Suspicious email reported by user', 'end_time': '2025-10-29T14:55:22.737Z', 'event_types': ['dissemination-phishing-emails'], 'name': 'Pote

## D.3 Create Event Sequence Objects

Create sequence objects that represent the workflow steps for the Event objects in proper sequential order.

In [75]:
# Create sequence objects following OLD notebook pattern
print("🔗 Creating Event sequence objects...")

# D.3.1 Create Event Alert Sequence (seq_E_1)
sequence_data_path = "SDO/Sequence/sequence_alert.json"
results_path = "step2/sequence_event_single.json"  # ✅ Added: Missing results path
step_type = "single_step"
sequence_type = "event"
sequenced_object = event_obj["id"]

seq_E_1 = invoke_make_sequence_block(
    sequence_data_path, 
    results_path, 
    step_type=step_type, 
    sequence_type=sequence_type, 
    sequenced_object=sequenced_object, 
    on_completion=None, 
    on_success=None, 
    on_failure=None, 
    next_steps=None
)

print(f"✅ Event alert sequence created (seq_E_1): {seq_E_1.get('id', 'Unknown')}")

# D.3.2 Create Event Start Sequence (seq_E_0)
sequence_data_path = "SDO/Sequence/sequence_start.json"
results_path = "step2/sequence_event_start.json"  # ✅ Added: Missing results path
step_type = "start_step"
sequence_type = "event"
on_completion = seq_E_1["id"]

seq_E_0 = invoke_make_sequence_block(
    sequence_data_path, 
    results_path, 
    step_type=step_type, 
    sequence_type=sequence_type, 
    sequenced_object=None, 
    on_completion=on_completion, 
    on_success=None, 
    on_failure=None, 
    next_steps=None
)

print(f"✅ Event start sequence created (seq_E_0): {seq_E_0.get('id', 'Unknown')}")

# Update sequence references for incident
sequence_start_refs.append(seq_E_0["id"])
sequence_refs.append(seq_E_1["id"])

🔗 Creating Event sequence objects...
{
    "type": "sequence",
    "spec_version": "2.1",
    "id": "sequence--5839f382-b0ee-4e31-863c-d6106f0c22a0",
    "created": "2025-10-29T14:55:22.931Z",
    "modified": "2025-10-29T14:55:22.931Z",
    "sequenced_object": "event--40cb3fd1-4092-47f0-93c7-cf75d9b18f64",
    "sequence_type": "event",
    "step_type": "single_step",
    "extensions": {
        "extension-definition--be0c7c79-1961-43db-afde-637066a87a64": {
            "extension_type": "new-sdo"
        }
    }
}
✅ Event alert sequence created (seq_E_1): sequence--5839f382-b0ee-4e31-863c-d6106f0c22a0
{
    "type": "sequence",
    "spec_version": "2.1",
    "id": "sequence--3c33d2a9-d378-4a75-bf92-1ead79b3d2e9",
    "created": "2025-10-29T14:55:22.980Z",
    "modified": "2025-10-29T14:55:22.980Z",
    "sequence_type": "event",
    "step_type": "start_step",
    "on_completion": "sequence--5839f382-b0ee-4e31-863c-d6106f0c22a0",
    "extensions": {
        "extension-definition--be0c7c79

## D.4 Create Task Sequence Objects

Create sequence objects that represent the workflow steps for the Task objects in proper sequential order.

In [76]:
# Create Task sequence objects following OLD notebook pattern
print("🔗 Creating Task sequence objects...")

# D.4.1 Create Task Alert Sequence (seq_T_1)
sequence_data_path = "SDO/Sequence/sequence_alert.json"
results_path = "step2/sequence_task_single.json"  # ✅ Added: Missing results path
step_type = "single_step"
sequence_type = "task"
sequenced_object = task_obj["id"]

seq_T_1 = invoke_make_sequence_block(
    sequence_data_path, 
    results_path, 
    step_type=step_type, 
    sequence_type=sequence_type, 
    sequenced_object=sequenced_object, 
    on_completion=None, 
    on_success=None, 
    on_failure=None, 
    next_steps=None
)

print(f"✅ Task alert sequence created (seq_T_1): {seq_T_1.get('id', 'Unknown')}")

# D.4.2 Create Task Start Sequence (seq_T_0)
sequence_data_path = "SDO/Sequence/sequence_start.json"
results_path = "step2/sequence_task_start.json"  # ✅ Added: Missing results path
step_type = "start_step"
sequence_type = "task"
on_completion = seq_T_1["id"]

seq_T_0 = invoke_make_sequence_block(
    sequence_data_path, 
    results_path, 
    step_type=step_type, 
    sequence_type=sequence_type, 
    sequenced_object=None, 
    on_completion=on_completion, 
    on_success=None, 
    on_failure=None, 
    next_steps=None
)

print(f"✅ Task start sequence created (seq_T_0): {seq_T_0.get('id', 'Unknown')}")

# Update sequence references for incident
sequence_start_refs.append(seq_T_0["id"])
sequence_refs.append(seq_T_1["id"])

print(f"📋 Complete sequence workflow created:")
print(f"   - Event sequences: {seq_E_0['id']} → {seq_E_1['id']}")
print(f"   - Task sequences: {seq_T_0['id']} → {seq_T_1['id']}")
print(f"   - Total sequence_start_refs: {len(sequence_start_refs)}")
print(f"   - Total sequence_refs: {len(sequence_refs)}")

🔗 Creating Task sequence objects...
{
    "type": "sequence",
    "spec_version": "2.1",
    "id": "sequence--adfb3fb5-d692-4590-8429-e1b0a2808734",
    "created": "2025-10-29T14:55:23.380Z",
    "modified": "2025-10-29T14:55:23.380Z",
    "sequenced_object": "task--359146be-e536-4a0d-99e7-2de98cf06715",
    "sequence_type": "task",
    "step_type": "single_step",
    "extensions": {
        "extension-definition--be0c7c79-1961-43db-afde-637066a87a64": {
            "extension_type": "new-sdo"
        }
    }
}
✅ Task alert sequence created (seq_T_1): sequence--adfb3fb5-d692-4590-8429-e1b0a2808734
{
    "type": "sequence",
    "spec_version": "2.1",
    "id": "sequence--480a3c4d-9d56-4406-b1d7-f90a534d25f5",
    "created": "2025-10-29T14:55:23.800Z",
    "modified": "2025-10-29T14:55:23.800Z",
    "sequence_type": "task",
    "step_type": "start_step",
    "on_completion": "sequence--adfb3fb5-d692-4590-8429-e1b0a2808734",
    "extensions": {
        "extension-definition--be0c7c79-1961

## D.5 Create Evidence Relationships

Create the "derived-from" relationship that links the email message to the malicious URL, following the OLD notebook pattern.

In [77]:
# Create "derived-from" relationship following OLD notebook pattern
print("🔗 Creating evidence derived-from relationship...")

# Configure relationship template and results path
sro_data_path = "SRO/Relationship/sro_derived.json"
sro_results_path = "step2/evidence_derived_relationship"

# Set up relationship parameters
relationship_type = "derived-from"

# Create derived-from relationship: email-message → url
sus_relation = invoke_sro_block(
    sro_data_path,
    sro_results_path,
    email_msg_obj,  # ✅ Fixed: Use correct variable name (email_msg_obj)
    malicious_url_obj,  # target object (URL)
    relationship_type
)

print(f"✅ Derived-from relationship created successfully")
print(f"   - Relationship ID: {sus_relation.get('id', 'Unknown')}")
print(f"   - Source: {email_msg_obj['id']} (email-message)")
print(f"   - Target: {malicious_url_obj['id']} (url)")
print(f"   - Type: {relationship_type}")
print(f"   - Context storage result: {sro_results_path}")

🔗 Creating evidence derived-from relationship...
input file->{'relationship_form': {'base_required': {'type': 'relationship', 'spec_version': '2.1', 'id': '', 'created': '', 'modified': ''}, 'base_optional': {'created_by_ref': '', 'revoked': None, 'labels': [], 'lang': '', 'external_references': [], 'object_marking_refs': [], 'granular_markings': [], 'defanged': None}, 'object': {'relationship_type': 'derived-from', 'source_ref': '', 'target_ref': '', 'start_time': None, 'stop_time': None}, 'extensions': {}, 'sub': {}}, 'source': {'type': 'email-message', 'spec_version': '2.1', 'id': 'email-message--6090e3d4-1fa8-5b36-9d2d-4a66d824995d', 'is_multipart': False, 'date': '2020-10-19T01:01:01Z', 'from_ref': 'email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf212', 'to_refs': ['email-addr--4722424c-7012-56b0-84d5-01d076fc547b'], 'subject': 'we are coming for you', 'body': 'some bad stuff written here'}, 'target': {'type': 'url', 'spec_version': '2.1', 'id': 'url--a824ef3f-83a0-51ae-9b5b-817580ec7

## E. Create Investigation Relationships

Establish STIX Relationship Objects (SROs) that link evidence objects to the incident and demonstrate attack patterns.

### E.1 Link Observables to Incident

Create relationships that connect the phishing evidence to the primary incident investigation.

In [78]:
# Configure evidence relationship creation - use working template
evidence_relationship_template = "SRO/Relationship/sro_derived.json"  # ✅ Use working template
evidence_relationship_results = "step2/evidence_relationships"

print(f"🔗 Creating evidence relationships for incident investigation")

# Define relationship type for evidence linking
relationship_type = "related-to"  # Evidence is related to the incident

# Create relationship linking email evidence to incident
email_relationship = invoke_sro_block(
    evidence_relationship_template,
    evidence_relationship_results + "_email",
    attacker_email_obj,
    incident_obj,
    relationship_type
)

# Create relationship linking URL evidence to incident
url_relationship = invoke_sro_block(
    evidence_relationship_template,
    evidence_relationship_results + "_url",
    malicious_url_obj,
    incident_obj,
    relationship_type
)

# NOTE: File relationship omitted due to file creation limitations
# file_relationship = invoke_sro_block(...)

# Configure context type for relationship storage
context_type = {
    "context_type": "unattached"  # ✅ Store as unattached like OLD notebook
}

# Save individual evidence relationships to incident context (follow OLD pattern)
email_rel_obj_path = results_base + evidence_relationship_results + "_email__rel.json"
email_rel_context_path = results_base + "step2/context/email_relationship_context.json"

result1 = invoke_save_incident_context_block(
    email_rel_obj_path,
    email_rel_context_path,
    context_type
)

url_rel_obj_path = results_base + evidence_relationship_results + "_url__rel.json"
url_rel_context_path = results_base + "step2/context/url_relationship_context.json"

result2 = invoke_save_incident_context_block(
    url_rel_obj_path,
    url_rel_context_path,
    context_type
)

print(f"✅ Evidence relationships created successfully")
print(f"   - Email → Incident relationship created: {result1}")
print(f"   - URL → Incident relationship created: {result2}")
print(f"   - File relationship omitted (template limitations)")

🔗 Creating evidence relationships for incident investigation
input file->{'relationship_form': {'base_required': {'type': 'relationship', 'spec_version': '2.1', 'id': '', 'created': '', 'modified': ''}, 'base_optional': {'created_by_ref': '', 'revoked': None, 'labels': [], 'lang': '', 'external_references': [], 'object_marking_refs': [], 'granular_markings': [], 'defanged': None}, 'object': {'relationship_type': 'derived-from', 'source_ref': '', 'target_ref': '', 'start_time': None, 'stop_time': None}, 'extensions': {}, 'sub': {}}, 'source': {'type': 'email-addr', 'spec_version': '2.1', 'id': 'email-addr--eb38d07e-6ba8-56c1-b107-d4db4aacf212', 'value': 'evil@northkorea.nk', 'display_name': 'Bad Man'}, 'target': {'type': 'incident', 'spec_version': '2.1', 'id': 'incident--d7aaa480-7a3e-4141-8fe0-baaeb6e93f24', 'created': '2025-10-29T14:55:21.889Z', 'modified': '2025-10-29T14:55:21.889Z', 'name': 'potential phishing', 'extensions': {'extension-definition--ef765651-680c-498d-9894-99799f2f

## F. Retrieve Reporter Information

Get the identity of the employee who reported the phishing email from the company context established in Step 1.

In [80]:
# Configure reporter identity retrieval from company context
print(f"👤 Retrieving reporter information: {phishing_scenario['reporter_email']}")

# Define search query for reporter email in company context
reporter_email_query = {
    "type": "email-addr",
    "property": {
        "path": ["value"],
        "source_value": phishing_scenario['reporter_email'],
        "comparator": "EQ"
    }
}

# Configure context type for company search
company_context_type = {
    "context_type": "users"  # Search in company users.json
}

# Search for reporter email in company context
try:
    reporter_email_obj = invoke_get_from_company_block(
        reporter_email_query,
        company_context_type,
        source_value=None,
        source_id=None
    )
    
    print(f"✅ Reporter email found in company context")
    print(f"   - Email ID: {reporter_email_obj.get('id', 'Not found')}")
    
    # If email found, get associated user account and identity
    if 'belongs_to_ref' in reporter_email_obj:
        account_id = reporter_email_obj['belongs_to_ref']
        print(f"   - Associated account: {account_id}")
    
except Exception as e:
    print(f"⚠️ Reporter not found in company context: {str(e)}")
    print(f"   - This is expected if company context was not properly established")
    print(f"   - Proceeding with incident creation without reporter linkage")

👤 Retrieving reporter information: alice.jones@company.com
company query->{'type': 'email-addr', 'property': {'path': ['value'], 'source_value': 'alice.jones@company.com', 'comparator': 'EQ'}}
✅ Reporter email found in company context
   - Email ID: email-addr--4722424c-7012-56b0-84d5-01d076fc547b
   - Associated account: user-account--597ad4d4-35ba-585d-8f6d-134a75032f9b


## 🎉 Phishing Incident Creation Complete!

Your phishing incident investigation has been successfully established with comprehensive evidence documentation:

### ✅ Created Incident Infrastructure
- **Primary Incident**: Phishing email investigation with unique incident ID
- **Evidence Objects**: Email address, malicious URL, and suspicious attachment documented
- **Threat Indicators**: Email domain and URL pattern indicators for detection
- **Investigation Relationships**: Evidence linked to incident for analysis
- **Context Storage**: All objects organized in incident-specific context

### 📁 Incident Context Structure
```
context_mem/
├── context_map.json                    # Updated with new incident
├── usr/                               # Personal user context
├── identity--{company-uuid}/           # Company context
└── incident--{incident-uuid}/          # ✅ NEW: Phishing incident context
    ├── incident.json                  # Primary incident object
    ├── observables.json               # Email, URL, file evidence
    ├── indicators.json                # Threat detection patterns
    └── relationships.json             # Evidence-incident linkages
```

### 🔍 Investigation Assets Created
- **Attacker Email**: `security-update@bankofamerica-verify.com` (spoofed sender)
- **Malicious URL**: `https://bankofamerica-verify.com/login-verify.php` (credential theft)
- **Suspicious Attachment**: `account_verification.pdf` (social engineering)
- **Domain Indicator**: Pattern for detecting similar attacks
- **URL Indicator**: Pattern for identifying credential theft attempts

### 🚀 Next Steps
You're now ready to:
1. **Analyze Evidence**: Examine collected observables for attack patterns
2. **Threat Hunting**: Use indicators to search for similar attacks
3. **Impact Assessment**: Determine scope and business impact
4. **Response Actions**: Implement containment and remediation
5. **Intelligence Sharing**: Export STIX objects for threat intelligence

### 🛡️ Incident Investigation Features
- **STIX 2.1 Compliance**: All objects follow industry standards
- **Dual-Layer Format**: Pure STIX data + UI visualization metadata
- **Context Isolation**: Incident data separate from other investigations
- **Evidence Tracking**: Complete chain of custody for forensic analysis
- **Relationship Modeling**: Clear connections between evidence and incident

Your phishing incident is now fully documented and ready for comprehensive cybersecurity investigation and response operations.

In [83]:
# Check sequence objects
print("Sequence E_0:", seq_E_0['id'] if seq_E_0 else "None")
print("Sequence E_1:", seq_E_1['id'] if seq_E_1 else "None") 
print("Sequence T_0:", seq_T_0['id'] if seq_T_0 else "None")
print("Sequence T_1:", seq_T_1['id'] if seq_T_1 else "None")

Sequence E_0: sequence--3c33d2a9-d378-4a75-bf92-1ead79b3d2e9
Sequence E_1: sequence--5839f382-b0ee-4e31-863c-d6106f0c22a0
Sequence T_0: sequence--480a3c4d-9d56-4406-b1d7-f90a534d25f5
Sequence T_1: sequence--adfb3fb5-d692-4590-8429-e1b0a2808734


In [82]:
# Check sighting object
print("Sighting object:", sighting_obj['id'] if sighting_obj else "None")
print("Sighting type:", type(sighting_obj))

Sighting object: sighting--1685385a-7862-4141-955d-f2f990c33ede
Sighting type: <class 'dict'>


In [84]:
# Find where sequences are created by checking their paths  
if 'seq_E_0' in locals():
    print("Found seq_E_0:", type(seq_E_0))
    if isinstance(seq_E_0, dict):
        print("Keys:", list(seq_E_0.keys()))
        if 'step_type' in seq_E_0:
            print("Step type:", seq_E_0['step_type'])
        if 'sequence_type' in seq_E_0:  
            print("Sequence type:", seq_E_0['sequence_type'])

Found seq_E_0: <class 'dict'>
Keys: ['type', 'spec_version', 'id', 'created', 'modified', 'sequence_type', 'step_type', 'on_completion', 'extensions']
Step type: start_step
Sequence type: event


In [85]:
# Print the sequence object details to find which files they were created from
for seq_name in ['seq_E_0', 'seq_E_1', 'seq_T_0', 'seq_T_1']:
    if seq_name in locals():
        seq_obj = locals()[seq_name]
        print(f"\n{seq_name}:")
        print(f"  ID: {seq_obj['id']}")
        print(f"  Step Type: {seq_obj['step_type']}")
        print(f"  Sequence Type: {seq_obj['sequence_type']}")
        if 'on_completion' in seq_obj:
            print(f"  On Completion: {seq_obj['on_completion']}")
        if 'sequenced_object' in seq_obj:
            print(f"  Sequenced Object: {seq_obj['sequenced_object']}")


seq_E_0:
  ID: sequence--3c33d2a9-d378-4a75-bf92-1ead79b3d2e9
  Step Type: start_step
  Sequence Type: event
  On Completion: sequence--5839f382-b0ee-4e31-863c-d6106f0c22a0

seq_E_1:
  ID: sequence--5839f382-b0ee-4e31-863c-d6106f0c22a0
  Step Type: single_step
  Sequence Type: event
  Sequenced Object: event--40cb3fd1-4092-47f0-93c7-cf75d9b18f64

seq_T_0:
  ID: sequence--480a3c4d-9d56-4406-b1d7-f90a534d25f5
  Step Type: start_step
  Sequence Type: task
  On Completion: sequence--adfb3fb5-d692-4590-8429-e1b0a2808734

seq_T_1:
  ID: sequence--adfb3fb5-d692-4590-8429-e1b0a2808734
  Step Type: single_step
  Sequence Type: task
  Sequenced Object: task--359146be-e536-4a0d-99e7-2de98cf06715


## F. Save Sequence Objects to Context Files

**CRITICAL FIX**: Save sequence objects to incident context files so they are available for promotion to incident context.

The `promote_objects.py` function expects sequence objects to be in:
- `/sequence_start_refs.json` for start_step sequences (seq_E_0, seq_T_0)  
- `/sequence_refs.json` for regular sequences (seq_E_1, seq_T_1)

Without these saves, sequences exist only in memory and are lost when the notebook ends.

In [86]:
# Save sequence objects to incident context files
print("🔧 CRITICAL FIX: Saving sequence objects to context files...")

# Create temporary files for sequences that need to be saved
sequence_temp_dir = results_base + "step2/sequences/"
os.makedirs(sequence_temp_dir, exist_ok=True)

# Function to save sequence object to temporary file then to context
def save_sequence_to_context(seq_obj, seq_name, context_type_name):
    # Save sequence to temporary file first
    temp_path = sequence_temp_dir + f"{seq_name}.json"
    with open(temp_path, 'w') as f:
        json.dump(seq_obj, f)
    
    # Setup context type for save
    context_type = {"context_type": context_type_name}
    
    # Save to incident context
    context_result_path = results_base + f"step2/context/{seq_name}_context.json"
    result = invoke_save_incident_context_block(temp_path, context_result_path, context_type)
    
    print(f"✅ {seq_name} saved to context type '{context_type_name}'")
    return result

# Save start_step sequences to "start" context (maps to sequence_start_refs.json)
save_sequence_to_context(seq_E_0, "seq_E_0", "start") 
save_sequence_to_context(seq_T_0, "seq_T_0", "start")

# Save regular sequences to "sequence" context (maps to sequence_refs.json)  
save_sequence_to_context(seq_E_1, "seq_E_1", "sequence")
save_sequence_to_context(seq_T_1, "seq_T_1", "sequence")

print("🎉 All sequence objects saved to incident context files!")
print("   - sequence_start_refs.json now contains seq_E_0 and seq_T_0")
print("   - sequence_refs.json now contains seq_E_1 and seq_T_1")

🔧 CRITICAL FIX: Saving sequence objects to context files...
input data->{'stix_object': {'type': 'sequence', 'spec_version': '2.1', 'id': 'sequence--3c33d2a9-d378-4a75-bf92-1ead79b3d2e9', 'created': '2025-10-29T14:55:22.980Z', 'modified': '2025-10-29T14:55:22.980Z', 'sequence_type': 'event', 'step_type': 'start_step', 'on_completion': 'sequence--5839f382-b0ee-4e31-863c-d6106f0c22a0', 'extensions': {'extension-definition--be0c7c79-1961-43db-afde-637066a87a64': {'extension_type': 'new-sdo'}}}, 'context_type': {'context_type': 'start'}}
from ports 
stix_object->{'type': 'sequence', 'spec_version': '2.1', 'id': 'sequence--3c33d2a9-d378-4a75-bf92-1ead79b3d2e9', 'created': '2025-10-29T14:55:22.980Z', 'modified': '2025-10-29T14:55:22.980Z', 'sequence_type': 'event', 'step_type': 'start_step', 'on_completion': 'sequence--5839f382-b0ee-4e31-863c-d6106f0c22a0', 'extensions': {'extension-definition--be0c7c79-1961-43db-afde-637066a87a64': {'extension_type': 'new-sdo'}}}
context type->start
✅ seq_E